In [27]:
import concurrent.futures
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
from src.dict_municipios import dict_municipios
import sys
sys.path.append("src")
from func import obtener_datos_municipio


<function func.obtener_datos_municipio(municipio, code, max_retries=3)>

In [2]:
def wait_and_click(drivers, xpath):
    """
    Espera a que un elemento esté disponible en el DOM, luego intenta hacer clic en él hasta 5 veces.

    Args:
        drivers (selenium.webdriver): El controlador de Selenium que gestiona la interacción con el navegador.
        xpath (str): La ruta xpath del elemento que se quiere encontrar y hacer clic.

    Returns:
        WebElement: El botón o elemento en el que se ha hecho clic.
    """
    tries = 0
    while True and tries < 5:
        try:
            button = WebDriverWait(drivers, 5).until(EC.presence_of_element_located(("xpath", xpath)))
            sleep(2)
            button.click()
            break
        except:
            tries += 1
            print("Try number", tries)
            continue
    return button

In [6]:
df = pd.read_csv("df_municipio_cat.csv", index_col=0)
municipios = df["Nombre_Municipio"].unique()

Método para sacar los municipios.

In [60]:
# dict_codes = dict()
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')

# for municipio in tqdm(municipios[41:]):
#     driver = webdriver.Chrome(options=options)
#     driver.maximize_window()
#     driver.get(url = "https://www.wunderground.com")
#     cookies_frame = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", "#sp_message_iframe_1165301")))
#     driver.switch_to.frame(cookies_frame)
#     reject_button = wait_and_click(driver, '//*[@id="notice"]/div[3]/div[2]/button')
#     sleep(1.5)
#     driver.switch_to.default_content()
#     search_box = wait_and_click(driver, xpath = '/html/body/app-root/app-home-page/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[1]/div[2]/lib-search/div/div/div/input')
#     search_box.send_keys(municipio+", Madrid")
#     WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="wuForm"]/search-autocomplete/ul')))
#     wait_and_click(driver, xpath='//*[@id="wuForm"]/search-autocomplete/ul/li[2]/a/span[1]')

#     wait_and_click(driver, xpath='//*[@id="inner-content"]/div[2]/lib-city-header/div[1]/div/div/a[1]')

#     code = driver.find_element("xpath", '//*[@id="inner-content"]/div[1]/app-dashboard-header/div[2]/div/div[2]/h1').text.split("-")[1].strip()
#     dict_codes[municipio] = code

 11%|█         | 6/57 [03:00<24:27, 28.78s/it]

Try number 1


 19%|█▉        | 11/57 [05:21<21:49, 28.46s/it]

Try number 1


 30%|██▉       | 17/57 [08:08<18:33, 27.84s/it]

...

 89%|████████▉ | 51/57 [24:27<02:45, 27.64s/it]

Try number 1


 98%|█████████▊| 56/57 [26:48<00:27, 27.92s/it]

Try number 1


100%|██████████| 57/57 [27:19<00:00, 28.76s/it]


Exportamos los códigos.

In [3]:
from src.dict_municipios import dict_municipios

dict_municipios

df_municipios2 = pd.DataFrame(list(dict_municipios.keys()), list(dict_municipios.values())).reset_index()

df_municipios2.to_csv("df_codigos.csv")

Convertimos a dataframe para el uso posterior.

In [4]:
df_municipios = pd.DataFrame(list(dict_municipios.keys()), list(dict_municipios.values()))
df_municipios = df_municipios.head(10)

Ahora definimos las funciones que van a estar ejecutándose durante los procesos paralelos.

In [29]:


df_final = pd.DataFrame()

max_workers = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    resultados = executor.map(obtener_datos_municipio, df_municipios[0], df_municipios.index)

for resultado in resultados:
    df_final = pd.concat([df_final, resultado], ignore_index=True)

df_final.head(10)

,Municipio,Date,High Temperature,Avg Temperature,Low Temperature,High Dew Point,Avg Dew Point,Low Dew Point,High Humidity,Avg Humidity,Low Humidity,High Speed,Avg Speed,Low Speed,High Pressure,Low Pressure,Sum
0,Ajalvir,10/1/2024,84.7,65.9,52.2,52.7,41.8,32.9,56,42,24,5.1,0.1,0.0,30.19,30.06,0.00
1,Ajalvir,10/2/2024,75.9,68.6,61.5,61.2,55.9,50.4,77,64,50,6.5,0.8,0.0,30.11,29.92,0.00
2,Ajalvir,10/3/2024,75.6,65.4,58.6,61.0,48.3,39.6,84,57,29,8.9,1.5,0.0,30.10,29.94,0.02
3,Ajalvir,10/4/2024,80.2,62.0,49.8,50.0,45.2,40.1,72,55,32,3.6,0.2,0.0,30.13,30.04,0.01
4,Ajalvir,10/5/2024,78.1,63.7,50.0,59.0,49.8,42.3,78,61,42,3.8,0.3,0.0,30.11,30.01,0.00
5,Ajalvir,10/6/2024,74.1,67.5,63.5,59.0,56.1,53.4,77,67,51,5.1,0.8,0.0,30.06,29.97,0.00
6,Ajalvir,10/7/2024,73.8,61.7,56.3,57.4,53.1,47.1,95,75,41,6.7,0.2,0.0,29.98,29.81,0.15
7,Ajalvir,10/8/2024,63.9,58.3,54.3,56.1,53.6,50.9,94,85,67,5.1,0.8,0.0,30.04,29.91,0.15
8,Ajalvir,10/9/2024,66.4,60.4,56.8,62.4,56.2,50.5,95,86,69,8.1,0.6,0.0,29.99,29.70,0.30
9,Ajalvir,10/10/2024,70.2,57.0,47.3,50.9,45.0,39.7,85,66,36,4.0,0.3,0.0,30.10,29.89,0.30


In [31]:
df_final.shape

(2909, 17)